Для курсового проекта я выбрала тему - "Heart Failure"
Для этого взяла дата-сет "Heart Failurer"
https://www.kaggle.com/fedesoriano/heart-failure-prediction

Я хочу обучить модель, которая по симптомам будет прогнозировать развитие злокачественной опухоли

In [73]:
import pandas as pd
import numpy as np
import dill
import seaborn as sns
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


import itertools
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score, \
        roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import matplotlib.pyplot as plt
import catboost as catb
%matplotlib inline

In [74]:
# input
DATASET_PATH = '../course_project/course_project_train.csv'

TEST_DATASET_PATH = 'heart.csv'




# output
TRAIN_X_FULL_PATH = 'hd_X_train.csv'
TRAIN_Y_FULL_PATH = "hd_y_train.csv"

TEST_X_FULL_PATH = "hd_X_test.csv"
TEST_Y_FULL_PATH = "hd_y_test.csv"

PREP_DATASET_PATH = '../course_project/project_train_data_prep.csv'

TRAIN_PART_PATH = '../course_project_train_part_b.csv'
TEST_PART_PATH = '../course_project_test_part.csv'

PIPELINE_FILE_PATH = "heart_decease_pipeline.dill"


In [75]:
df = pd.read_csv(TEST_DATASET_PATH)
df.head(3)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0


In [76]:
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


In [77]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


В качестве целевой переменной мы возьмем HeartDisease - . 

In [78]:
df['HeartDisease'].value_counts()

1    508
0    410
Name: HeartDisease, dtype: int64

Наши классы достаточно сбалансированы

### Обзор переменных

In [79]:
df['Sex'].value_counts()

M    725
F    193
Name: Sex, dtype: int64

In [80]:
df['ChestPainType'].value_counts()

ASY    496
NAP    203
ATA    173
TA      46
Name: ChestPainType, dtype: int64

In [81]:
df['RestingECG'].value_counts()

Normal    552
LVH       188
ST        178
Name: RestingECG, dtype: int64

In [82]:
df['ExerciseAngina'].value_counts()

N    547
Y    371
Name: ExerciseAngina, dtype: int64

In [83]:
df['ST_Slope'].value_counts()

Flat    460
Up      395
Down     63
Name: ST_Slope, dtype: int64

In [84]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


На основе FeatureImportance пробной модели сократим количеством признаков, на которых обучается модель. уберем RestingECG, RestingBP, MaxHR, FastingBS

In [85]:
df.drop(axis=1, columns = ['RestingECG', 'RestingBP', 'MaxHR', 'FastingBS'], inplace=True)

In [90]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int32  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   Cholesterol     918 non-null    int32  
 4   ExerciseAngina  918 non-null    object 
 5   Oldpeak         918 non-null    float64
 6   ST_Slope        918 non-null    object 
 7   HeartDisease    918 non-null    int64  
dtypes: float64(1), int32(2), int64(1), object(4)
memory usage: 50.3+ KB


In [91]:
df["Age"] = df["Age"].astype('int32')
df["Cholesterol"] = df["Cholesterol"].astype('int32')
df["HeartDisease"] = df["HeartDisease"].astype('int32')

In [92]:
cat_features = ['Sex','ChestPainType', 'ExerciseAngina', 'ST_Slope']

In [93]:
continuous_features = ['Age','Cholesterol','Oldpeak']

In [94]:
target = 'HeartDisease'

In [95]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

#соберем наш простой pipeline, но нам понадобится написать класс для выбора нужного поля
class FeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, column):
        self.column = column

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.column]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in self.columns:
            if col_ not in test_columns:
                X[col_] = 0
        return X[self.columns]

In [96]:
final_transformers = list()

for cat_col in cat_features:
    cat_transformer = Pipeline([
                ('selector', FeatureSelector(column=cat_col)),
                ('ohe', OHEEncoder(key=cat_col))
            ])
    final_transformers.append((cat_col, cat_transformer))
    
for cont_col in continuous_features:
    cont_transformer = Pipeline([
                ('selector', NumberSelector(key=cont_col)),
                ('scaler', StandardScaler())
            ])
    final_transformers.append((cont_col, cont_transformer))

Объединим все это в единый пайплайн

In [97]:
feats = FeatureUnion(final_transformers)

feature_processing = Pipeline([('feats', feats)])

Теперь у нас есть пайплайн, который готовит признаки для моделирования.

Добавим модель. В первом варианте будет использовать RandomForestClassifier

In [98]:
from sklearn.ensemble import RandomForestClassifier

pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(random_state = 42)),
])

Теперь разделим наш датасет на тестовую и тренировочную выборки

In [99]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=[target]), df[target],test_size=0.25,
                               shuffle=True, random_state=42)


In [100]:
#save test
X_test.to_csv(TEST_X_FULL_PATH, index=None)
y_test.to_csv(TEST_Y_FULL_PATH, index=None)
#save train
X_train.to_csv(TRAIN_X_FULL_PATH, index=None)
y_train.to_csv(TRAIN_Y_FULL_PATH, index=None)


### Обучение модели

In [101]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [102]:
from sklearn.model_selection import GridSearchCV

params_rf={'classifier__max_features':[0.3, 0.5, 0.7],
        'classifier__min_samples_leaf':[1, 2, 3],
        'classifier__max_depth':[3,5,7]
        }

In [103]:
%time
grid_rf = GridSearchCV(pipeline_1,
                    param_grid=params_rf,
                    cv=6,
                    refit=False)

search_rf = grid_rf.fit(X_train, y_train)
search_rf.best_params_

CPU times: user 6 µs, sys: 2 µs, total: 8 µs
Wall time: 14.1 µs


{'classifier__max_depth': 7,
 'classifier__max_features': 0.3,
 'classifier__min_samples_leaf': 1}

In [104]:
pipeline_1 = Pipeline([
    ('features',feats),
    ('classifier', RandomForestClassifier(max_depth = 7,\
                                          max_features=0.3, \
                                          random_state = 42,\
                                         min_samples_leaf =2)),
])

In [105]:
#обучим наш пайплайн
pipeline_1.fit(X_train, y_train)

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


В качестве второго варианта для модели возьмем CatBoost

In [106]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(random_state = 42)),
])

In [107]:
params_cb={'classifier__iterations':[50, 100, 200],
        'classifier__max_depth':[3,5,7]
        }

In [108]:
%time
grid_cb = GridSearchCV(pipeline_2,
                    param_grid=params_cb,
                    cv=6,
                    refit=False)

search_cb = grid_cb.fit(X_train, y_train)


CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 10 µs
Learning rate set to 0.126681
0:	learn: 0.6396506	total: 1.37ms	remaining: 67.1ms
1:	learn: 0.5999934	total: 2.37ms	remaining: 56.9ms
2:	learn: 0.5612588	total: 3.48ms	remaining: 54.4ms
3:	learn: 0.5313902	total: 4.59ms	remaining: 52.8ms
4:	learn: 0.5058978	total: 5.78ms	remaining: 52ms
5:	learn: 0.4860302	total: 7.02ms	remaining: 51.5ms
6:	learn: 0.4696593	total: 8.53ms	remaining: 52.4ms
7:	learn: 0.4533983	total: 9.43ms	remaining: 49.5ms
8:	learn: 0.4384939	total: 10.4ms	remaining: 47.4ms
9:	learn: 0.4269121	total: 11.3ms	remaining: 45.1ms
10:	learn: 0.4148465	total: 12.1ms	remaining: 42.9ms
11:	learn: 0.4075709	total: 12.9ms	remaining: 40.8ms
12:	learn: 0.3992330	total: 13.8ms	remaining: 39.1ms
13:	learn: 0.3926396	total: 14.6ms	remaining: 37.6ms
14:	learn: 0.3832423	total: 15.5ms	remaining: 36.3ms
15:	learn: 0.3768977	total: 16.5ms	remaining: 35ms
16:	learn: 0.3713167	total: 17.4ms	remaining: 33.7ms
17:	learn: 0.3676245	

14:	learn: 0.3708873	total: 23.7ms	remaining: 55.4ms
15:	learn: 0.3644806	total: 27.1ms	remaining: 57.5ms
16:	learn: 0.3595830	total: 28.8ms	remaining: 56ms
17:	learn: 0.3559535	total: 30.1ms	remaining: 53.5ms
18:	learn: 0.3526625	total: 31.1ms	remaining: 50.8ms
19:	learn: 0.3497199	total: 34.6ms	remaining: 51.9ms
20:	learn: 0.3433566	total: 35.8ms	remaining: 49.4ms
21:	learn: 0.3401745	total: 37.9ms	remaining: 48.3ms
22:	learn: 0.3375311	total: 39.2ms	remaining: 46ms
23:	learn: 0.3353285	total: 40.3ms	remaining: 43.7ms
24:	learn: 0.3322452	total: 41.6ms	remaining: 41.6ms
25:	learn: 0.3303920	total: 43.5ms	remaining: 40.1ms
26:	learn: 0.3287473	total: 44.9ms	remaining: 38.3ms
27:	learn: 0.3272092	total: 46.9ms	remaining: 36.9ms
28:	learn: 0.3255314	total: 48.2ms	remaining: 34.9ms
29:	learn: 0.3229432	total: 49.1ms	remaining: 32.7ms
30:	learn: 0.3211961	total: 50.4ms	remaining: 30.9ms
31:	learn: 0.3195172	total: 51.4ms	remaining: 28.9ms
32:	learn: 0.3187446	total: 52.5ms	remaining: 27ms

10:	learn: 0.4038017	total: 33ms	remaining: 117ms
11:	learn: 0.3938854	total: 37.9ms	remaining: 120ms
12:	learn: 0.3856170	total: 40.8ms	remaining: 116ms
13:	learn: 0.3783545	total: 42.9ms	remaining: 110ms
14:	learn: 0.3725941	total: 47.2ms	remaining: 110ms
15:	learn: 0.3664265	total: 49.3ms	remaining: 105ms
16:	learn: 0.3623415	total: 50.9ms	remaining: 98.8ms
17:	learn: 0.3562532	total: 52.8ms	remaining: 93.9ms
18:	learn: 0.3487788	total: 54.8ms	remaining: 89.4ms
19:	learn: 0.3451301	total: 56.8ms	remaining: 85.2ms
20:	learn: 0.3408884	total: 58.9ms	remaining: 81.3ms
21:	learn: 0.3373680	total: 61ms	remaining: 77.6ms
22:	learn: 0.3351925	total: 63.1ms	remaining: 74.1ms
23:	learn: 0.3320653	total: 66ms	remaining: 71.5ms
24:	learn: 0.3281925	total: 68.2ms	remaining: 68.2ms
25:	learn: 0.3252050	total: 70.2ms	remaining: 64.8ms
26:	learn: 0.3209309	total: 71.9ms	remaining: 61.3ms
27:	learn: 0.3177527	total: 74.2ms	remaining: 58.3ms
28:	learn: 0.3156064	total: 76.7ms	remaining: 55.6ms
29:	l

17:	learn: 0.3434342	total: 40.9ms	remaining: 72.7ms
18:	learn: 0.3373917	total: 45ms	remaining: 73.4ms
19:	learn: 0.3330789	total: 48ms	remaining: 72.1ms
20:	learn: 0.3285720	total: 50.4ms	remaining: 69.6ms
21:	learn: 0.3232988	total: 52.3ms	remaining: 66.5ms
22:	learn: 0.3192491	total: 54.3ms	remaining: 63.8ms
23:	learn: 0.3159910	total: 56.1ms	remaining: 60.7ms
24:	learn: 0.3122114	total: 57.8ms	remaining: 57.8ms
25:	learn: 0.3085467	total: 60.4ms	remaining: 55.8ms
26:	learn: 0.3057111	total: 63.1ms	remaining: 53.7ms
27:	learn: 0.3030011	total: 64.9ms	remaining: 51ms
28:	learn: 0.3010427	total: 66.8ms	remaining: 48.4ms
29:	learn: 0.2988704	total: 68.6ms	remaining: 45.7ms
30:	learn: 0.2961046	total: 70.5ms	remaining: 43.2ms
31:	learn: 0.2944002	total: 72.3ms	remaining: 40.7ms
32:	learn: 0.2920297	total: 74.1ms	remaining: 38.2ms
33:	learn: 0.2900151	total: 75.7ms	remaining: 35.6ms
34:	learn: 0.2875743	total: 77.4ms	remaining: 33.2ms
35:	learn: 0.2862670	total: 78.9ms	remaining: 30.7ms

Learning rate set to 0.126681
0:	learn: 0.6433840	total: 3.39ms	remaining: 166ms
1:	learn: 0.6006139	total: 5.76ms	remaining: 138ms
2:	learn: 0.5651881	total: 8.05ms	remaining: 126ms
3:	learn: 0.5355590	total: 10.9ms	remaining: 125ms
4:	learn: 0.5085302	total: 13.4ms	remaining: 121ms
5:	learn: 0.4860885	total: 15.9ms	remaining: 117ms
6:	learn: 0.4652441	total: 18.3ms	remaining: 112ms
7:	learn: 0.4493690	total: 20.4ms	remaining: 107ms
8:	learn: 0.4339272	total: 22.8ms	remaining: 104ms
9:	learn: 0.4192048	total: 24.9ms	remaining: 99.8ms
10:	learn: 0.4048501	total: 27.7ms	remaining: 98.2ms
11:	learn: 0.3939617	total: 31.2ms	remaining: 98.7ms
12:	learn: 0.3836503	total: 33.4ms	remaining: 95.2ms
13:	learn: 0.3752187	total: 35.8ms	remaining: 92ms
14:	learn: 0.3674999	total: 38ms	remaining: 88.7ms
15:	learn: 0.3595251	total: 40.8ms	remaining: 86.7ms
16:	learn: 0.3529535	total: 43.7ms	remaining: 84.9ms
17:	learn: 0.3456179	total: 46.3ms	remaining: 82.3ms
18:	learn: 0.3399516	total: 48.3ms	rema

...

60:	learn: 0.2434807	total: 186ms	remaining: 423ms
61:	learn: 0.2412719	total: 191ms	remaining: 425ms
62:	learn: 0.2399345	total: 195ms	remaining: 425ms
63:	learn: 0.2375229	total: 198ms	remaining: 422ms
64:	learn: 0.2355618	total: 201ms	remaining: 418ms
65:	learn: 0.2338811	total: 204ms	remaining: 415ms
66:	learn: 0.2326963	total: 208ms	remaining: 413ms
67:	learn: 0.2309684	total: 211ms	remaining: 409ms
68:	learn: 0.2294545	total: 213ms	remaining: 405ms
69:	learn: 0.2281818	total: 216ms	remaining: 402ms
70:	learn: 0.2271151	total: 222ms	remaining: 403ms
71:	learn: 0.2256109	total: 225ms	remaining: 400ms
72:	learn: 0.2247425	total: 227ms	remaining: 395ms
73:	learn: 0.2232576	total: 233ms	remaining: 398ms
74:	learn: 0.2224195	total: 236ms	remaining: 393ms
75:	learn: 0.2215530	total: 238ms	remaining: 388ms
76:	learn: 0.2202753	total: 240ms	remaining: 384ms
77:	learn: 0.2190659	total: 243ms	remaining: 379ms
78:	learn: 0.2172007	total: 245ms	remaining: 375ms
79:	learn: 0.2160054	total: 247

60:	learn: 0.2371465	total: 182ms	remaining: 416ms
61:	learn: 0.2352602	total: 187ms	remaining: 415ms
62:	learn: 0.2338063	total: 190ms	remaining: 413ms
63:	learn: 0.2322150	total: 194ms	remaining: 412ms
64:	learn: 0.2311687	total: 197ms	remaining: 409ms
65:	learn: 0.2289354	total: 200ms	remaining: 405ms
66:	learn: 0.2269634	total: 202ms	remaining: 402ms
67:	learn: 0.2251411	total: 205ms	remaining: 398ms
68:	learn: 0.2232304	total: 212ms	remaining: 403ms
69:	learn: 0.2216579	total: 215ms	remaining: 400ms
70:	learn: 0.2209854	total: 218ms	remaining: 397ms
71:	learn: 0.2191201	total: 222ms	remaining: 395ms
72:	learn: 0.2174836	total: 227ms	remaining: 395ms
73:	learn: 0.2161464	total: 230ms	remaining: 392ms
74:	learn: 0.2147395	total: 234ms	remaining: 390ms
75:	learn: 0.2138383	total: 237ms	remaining: 386ms
76:	learn: 0.2126020	total: 241ms	remaining: 385ms
77:	learn: 0.2109924	total: 244ms	remaining: 382ms
78:	learn: 0.2100329	total: 247ms	remaining: 378ms
79:	learn: 0.2085406	total: 250

64:	learn: 0.2298619	total: 187ms	remaining: 389ms
65:	learn: 0.2280235	total: 191ms	remaining: 387ms
66:	learn: 0.2259297	total: 195ms	remaining: 387ms
67:	learn: 0.2253797	total: 200ms	remaining: 388ms
68:	learn: 0.2246752	total: 202ms	remaining: 384ms
69:	learn: 0.2231618	total: 205ms	remaining: 382ms
70:	learn: 0.2212612	total: 210ms	remaining: 381ms
71:	learn: 0.2206794	total: 215ms	remaining: 382ms
72:	learn: 0.2191709	total: 218ms	remaining: 379ms
73:	learn: 0.2182692	total: 221ms	remaining: 376ms
74:	learn: 0.2176176	total: 229ms	remaining: 381ms
75:	learn: 0.2170086	total: 232ms	remaining: 379ms
76:	learn: 0.2153016	total: 235ms	remaining: 375ms
77:	learn: 0.2146423	total: 238ms	remaining: 372ms
78:	learn: 0.2124639	total: 241ms	remaining: 369ms
79:	learn: 0.2117054	total: 244ms	remaining: 366ms
80:	learn: 0.2103966	total: 247ms	remaining: 362ms
81:	learn: 0.2099753	total: 249ms	remaining: 358ms
82:	learn: 0.2091898	total: 251ms	remaining: 354ms
83:	learn: 0.2079643	total: 255

72:	learn: 0.2278229	total: 243ms	remaining: 422ms
73:	learn: 0.2265062	total: 249ms	remaining: 425ms
74:	learn: 0.2250205	total: 255ms	remaining: 425ms
75:	learn: 0.2236040	total: 259ms	remaining: 423ms
76:	learn: 0.2220876	total: 263ms	remaining: 419ms
77:	learn: 0.2209399	total: 274ms	remaining: 429ms
78:	learn: 0.2190283	total: 292ms	remaining: 447ms
79:	learn: 0.2189217	total: 301ms	remaining: 451ms
80:	learn: 0.2176395	total: 307ms	remaining: 451ms
81:	learn: 0.2165425	total: 312ms	remaining: 449ms
82:	learn: 0.2153061	total: 317ms	remaining: 447ms
83:	learn: 0.2139340	total: 323ms	remaining: 445ms
84:	learn: 0.2133738	total: 328ms	remaining: 444ms
85:	learn: 0.2130705	total: 332ms	remaining: 439ms
86:	learn: 0.2127898	total: 335ms	remaining: 436ms
87:	learn: 0.2119148	total: 339ms	remaining: 432ms
88:	learn: 0.2112009	total: 343ms	remaining: 428ms
89:	learn: 0.2102746	total: 346ms	remaining: 423ms
90:	learn: 0.2093146	total: 349ms	remaining: 418ms
91:	learn: 0.2083127	total: 351

48:	learn: 0.2510356	total: 190ms	remaining: 585ms
49:	learn: 0.2493128	total: 197ms	remaining: 592ms
50:	learn: 0.2459140	total: 203ms	remaining: 594ms
51:	learn: 0.2455984	total: 207ms	remaining: 588ms
52:	learn: 0.2435579	total: 213ms	remaining: 591ms
53:	learn: 0.2415956	total: 217ms	remaining: 586ms
54:	learn: 0.2405086	total: 223ms	remaining: 588ms
55:	learn: 0.2384591	total: 229ms	remaining: 589ms
56:	learn: 0.2369138	total: 233ms	remaining: 585ms
57:	learn: 0.2349998	total: 244ms	remaining: 597ms
58:	learn: 0.2332549	total: 247ms	remaining: 591ms
59:	learn: 0.2318517	total: 251ms	remaining: 585ms
60:	learn: 0.2295282	total: 255ms	remaining: 581ms
61:	learn: 0.2283020	total: 258ms	remaining: 575ms
62:	learn: 0.2276024	total: 262ms	remaining: 570ms
63:	learn: 0.2259531	total: 265ms	remaining: 564ms
64:	learn: 0.2238448	total: 274ms	remaining: 568ms
65:	learn: 0.2221433	total: 277ms	remaining: 563ms
66:	learn: 0.2214228	total: 281ms	remaining: 557ms
67:	learn: 0.2209731	total: 285

In [109]:
search_cb.best_params_

{'classifier__iterations': 200, 'classifier__max_depth': 3}

In [110]:

pipeline_2 = Pipeline([
    ('features',feats),
    ('classifier', catb.CatBoostClassifier(iterations = 200,\
                                           max_depth=3,
                                           random_state = 42)),
])


In [111]:
pipeline_2.fit(X_train, y_train)

Learning rate set to 0.038418
0:	learn: 0.6648744	total: 2.69ms	remaining: 534ms
1:	learn: 0.6456676	total: 4.74ms	remaining: 469ms
2:	learn: 0.6242099	total: 6.24ms	remaining: 410ms
3:	learn: 0.6031473	total: 7.94ms	remaining: 389ms
4:	learn: 0.5828988	total: 10.2ms	remaining: 398ms
5:	learn: 0.5629649	total: 15.7ms	remaining: 507ms
6:	learn: 0.5437890	total: 17.3ms	remaining: 478ms
7:	learn: 0.5327356	total: 18.9ms	remaining: 453ms
8:	learn: 0.5207788	total: 20.9ms	remaining: 443ms
9:	learn: 0.5051044	total: 22.5ms	remaining: 428ms
10:	learn: 0.4957560	total: 25.8ms	remaining: 444ms
11:	learn: 0.4860590	total: 27.9ms	remaining: 438ms
12:	learn: 0.4752287	total: 29.7ms	remaining: 427ms
13:	learn: 0.4649557	total: 31.3ms	remaining: 416ms
14:	learn: 0.4578381	total: 32.7ms	remaining: 404ms
15:	learn: 0.4501429	total: 34.1ms	remaining: 393ms
16:	learn: 0.4403168	total: 35.5ms	remaining: 382ms
17:	learn: 0.4353034	total: 36.9ms	remaining: 373ms
18:	learn: 0.4276963	total: 38.5ms	remaining

Pipeline(steps=[('features',
                 FeatureUnion(transformer_list=[('Sex',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='Sex')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='Sex'))])),
                                                ('ChestPainType',
                                                 Pipeline(steps=[('selector',
                                                                  FeatureSelector(column='ChestPainType')),
                                                                 ('ohe',
                                                                  OHEEncoder(key='ChestPainType'))])),
                                                ('ExerciseAngina',
                                                 Pipeline(steps=[('selector',


In [112]:
predictions1 = pipeline_1.predict_proba(X_test)
predictions2 = pipeline_2.predict_proba(X_test)

In [113]:
def get_model_scores(y_test, y_score):
    b=1
    precision, recall, thresholds = precision_recall_curve(y_test.values, y_score)
    fscore = (1+b**2)*(precision * recall) / (b**2*precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    roc_auc = roc_auc_score(y_test, y_score)
    return precision[ix], recall[ix], fscore[ix], roc_auc, thresholds[ix]

In [114]:
pr1, rec1, f1, roc1, thresh1 =  get_model_scores(y_test, predictions1[:, 1][:])
pr1, rec1, f1, roc1, thresh1 

(0.8854961832061069,
 0.8787878787878788,
 0.8821292775665399,
 0.9221938775510203,
 0.4288438592790484)

In [115]:
pr2, rec2, f2, roc2, thresh2 =  get_model_scores(y_test, predictions2[:, 1][:])
pr2, rec2, f2, roc2, thresh2 

(0.8776978417266187,
 0.9242424242424242,
 0.9003690036900369,
 0.9355674087816945,
 0.33601411138028925)

Выбираем pipeline_2(catboost), потому что у него метрики более высокие

In [116]:
def show_feature_importances(feature_names, feature_importances, get_top=None):
    feature_importances = pd.DataFrame({'feature': feature_names, 'importance': feature_importances})
    feature_importances = feature_importances.sort_values('importance', ascending=False)
       
    plt.figure(figsize = (20, len(feature_importances) * 0.355))
    
    sns.barplot(feature_importances['importance'], feature_importances['feature'])
    
    plt.xlabel('Importance')
    plt.title('Importance of features')
    plt.show()
    
    if get_top is not None:
        return feature_importances['feature'][:get_top].tolist()

На основе FeatureImportance сократим количеством признаков, на которых обучается модель.
уберем RestingECG, RestingBP, MaxHR, FastingBS

In [117]:
model_dict = {
    'model':pipeline_2,\
    'version':'0.0.1'
    
}

In [118]:
with open(PIPELINE_FILE_PATH, "wb") as f:
    dill.dump(model_dict, f)